# Vetorização dos CSV

## Setup

### Imports


In [ ]:
import chromadb
import uuid
import csv

### Estabelecendo conexão com o chromadb

In [ ]:
client = chromadb.Client()

### Criando/Carregando Collections

In [ ]:
nutrition = client.get_or_create_collection(name="nutrition")

### Carregando Arquivos

#### Tabela TACO

In [ ]:
taco_rows = []

with open("../../../docs/datasets/formattedTACO.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)  # lê como dicionários
    for row in reader:
        # transforma a linha em texto descritivo
        taco_rows.append(row)

taco_chunks = []

for row in taco_rows:
    chunk = ""
    #for key in row.keys():
    #    chunk += " " + str(key) # chave
    #    chunk += ": " + str(row[key]) + "," # valor
    chunk += f"O alimento {row['nome']} tem {row['umidade_(%)']}% de umidade, {row['energia_(kcal)']}kcal de energia, {row['energia_(kj)']}kj de energia, {row['proteina_(g)']}g de proteína, {row['lipideos_(g)']}g de lipídeos, {row['colesterol_(mg)']}mg de colesterol, {row['carboidrato_(g)']}g de carboídrato, {row['fibra_alimentar_(g)']}g de fibra alimentar, {row['cinzas_(g)']}g de cinzas, {row['calcio_(mg)']}mg de cálcio e {row['magnesio_(mg)']}mg de magnésio"
    taco_chunks.append(chunk)

print(taco_chunks)

#### Tabela TBCA

In [ ]:
tbca_rows = []

with open("../../../docs/datasets/formattedTBCA.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter=";")  # lê como dicionários
    for row in reader:
        # transforma a linha em texto descritivo
        tbca_rows.append(row)

tbca_chunks = []

for row in tbca_rows:
    chunk = ""
    #for key in row.keys():
    #    chunk += " " + str(key) # chave
    #    chunk += ": " + str(row[key]) + "," # valor
    chunk += f"O alimento {row['alimento'].replace(",", "").strip()} tem {row['energia_kj']}kj de energia, {row['energia_kcal']}kcal de energia, {row['umidade_g']}g de umidade, {row['carboidrato_total']}g de carboidrato total, {row['carboidrato_disponivel']}g de carboidrato disponivel, {row['proteina']}g de proteina, {row['lipidios']}g de lipidios, {row['fibra_alimentar']}g de fibra alimentar, {row['alcool']}g de alcool, {row['cinzas']}g de cinzas, {row['colesterol']}mg de colesterol, {row['acidos_graxos_saturados']}g de acidos graxos saturados, {row['acidos_graxos_monoinsaturados']}g de acidos graxos monoinsaturados, {row['acidos_graxos_polinsaturados']}g de acidos graxos polinsaturados, {row['acidos_graxos_trans']}g de acidos graxos trans, {row['calcio']}mg de calcio, {row['ferro']}mg de ferro, {row['sodio']}mg de sodio, {row['magnesio']}mg de magnesio, {row['fosforo']}mg de fosforo, {row['potassio']}mg de potassio, {row['zinco']}mg de zinco, {row['cobre']}mg de cobre, {row['selenio']}mcg de selenio, {row['vitamina_a_(re)']}mcg de vitamina a(re), {row['vitamina_a_(rae)']}mcg de vitamina a(rae), {row['vitamina_d']}mcg de vitamina d, {row['alfa-tocoferol_(vitamina_e)']}mg de alfa tocoferol ou vitamina e, {row['tiamina']}mg de tiamina, {row['riboflavina']}mg de riboflavina, {row['niacina']}mg de niacina, {row['vitamina_b6']}mg de vitamina b6, {row['vitamina_b12']}mcg de vitamina b12, {row['vitamina_c']}mg de vitamina c, {row['equivalente_de_folato']}mcg de equivalente de folato, {row['sal_de_adicao']}g de sal de adicao, {row['acucar_de_adicao']}g de acucar de adicao"
    tbca_chunks.append(chunk)

print(tbca_chunks[0])

### Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

taco_embeddings = model.encode(taco_chunks, convert_to_numpy=True)
print(f"Embeddings gerados: {taco_embeddings.shape}")
tbca_embeddings = model.encode(tbca_chunks, convert_to_numpy=True)
print(f"Embeddings gerados: {tbca_embeddings.shape}")

### Adicionando coleções

In [ ]:
# TACO
nutrition.add(
    ids=[str(uuid.uuid4()) for _ in taco_chunks],
    documents=taco_chunks,
    embeddings=taco_embeddings,
    metadatas=[{"origem":"taco"} for _ in taco_chunks]
)
# TBCA
nutrition.add(
    ids=[str(uuid.uuid4()) for _ in tbca_chunks],
    documents=tbca_chunks,
    embeddings=tbca_embeddings,
    metadatas=[{"origem": "tbca"} for _ in tbca_chunks]
)

### Queries de teste

In [ ]:
#import numpy as np

results = nutrition.query(
    query_texts=[
        "Quanto de proteína tem o arroz integral cru?",
        "Quanto de carboidrato tem o biscoito doce maisena?",
        "Qual alimento possui mais magnésio?",
        "Qual alimento tem maior fibra alimentar?"
    ],
    n_results=5,
    # where={"origem": "taco"}
)

for i, query_results in enumerate(results["documents"]):
    print(f"\nQuery {i}")
    print("\n".join(query_results))